Equal-Weight S&P 500 Index Fund
The goal is to create a python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal weight version of the index fund.  

In [1]:
#Import libraries
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math 

Import Stocks ~
I used a download of the S&P 500 rather than using an API because this method was free

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [3]:
# API token
from API_KEY import IEX_CLOUD_API_TOKEN

In [4]:
my_columns= ['Ticker','Stock Price', 'Market Capitalization','Number of Shares to Buy']
#final_dataframe = pd.DataFrame(columns = my_columns)

#Gonna run super slow
# for stock in stocks['Ticker'][:5]:
#     api_url = f'https://api.iex.cloud/v1/data/core/quote/{stock}?token={IEX_CLOUD_API_TOKEN}'
#     data = requests.get(api_url).json() 
#     d = data[0]
#     final_dataframe = final_dataframe._append(
#     pd.Series(
#     [
#         stock,
#         d['latestPrice'],
#         d['marketCap'],
#         'N/A'
#     ],
#         index = my_columns
#     ),
#     ignore_index = True
# )


In [5]:
def chunks(lst,n):
    """Yield succesive n-sized chunks from list."""
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [6]:
#The actual batch call
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_dataframe = pd.DataFrame(columns = my_columns)      
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        for d in data :
            if d['symbol'] == symbol:
                new_row = pd.Series([symbol, d['latestPrice'], d['marketCap'], 'N/A'], index = my_columns)
        final_dataframe = pd.concat([final_dataframe,new_row.to_frame().T], ignore_index=True)
        
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,130.54,38248755475,N/A
1,AAL,13.21,8633270666,N/A
2,AAP,61.3,3647471987,N/A
3,AAPL,185.92,2874675704320,N/A
4,ABBV,162.4,292320000000,N/A
...,...,...,...,...
500,YUM,129.13,36196200319,N/A
501,ZBH,123.07,25719256103,N/A
502,ZBRA,248.87,12781864647,N/A
503,ZION,42.11,6238545926,N/A


In [7]:
#Calculating shares to buy
portfolio_size = input('Enter the value of your portfolio: ')
try:
    val = float(portfolio_size)
except ValueError:
    print('Thats not a number! \nPlease try again:')
    portfolio_size = input('Enter the value of your portfolio: ')
    val = float(portfolio_size)

Enter the value of your portfolio: 1000000000


In [8]:
position_size = val/len(final_dataframe.index)

# Calculation of amount of shares to buy
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i,'Stock Price'] )
    

In [9]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,130.54,38248755475,15169
1,AAL,13.21,8633270666,149901
2,AAP,61.3,3647471987,32303
3,AAPL,185.92,2874675704320,10650
4,ABBV,162.4,292320000000,12193
...,...,...,...,...
500,YUM,129.13,36196200319,15334
501,ZBH,123.07,25719256103,16090
502,ZBRA,248.87,12781864647,7956
503,ZION,42.11,6238545926,47024


In [10]:
#Initializing xlxs writer
writer = pd.ExcelWriter('recommendeded_trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

In [11]:
background_color = "#0a0a23"
font_color = "#ffffff"

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [12]:
# writer.sheets['Recommended Trades'].set_column('A:A',18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B',18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C',18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D',18, string_format)
# writer.close()



In [14]:
column_formats = {
    'A' : ['Ticker', string_format],
    'B' : ['Stock Price', dollar_format],
    'C' : ['Market Capitalization', integer_format],
    'D' : ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}',20,column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1',column_formats[column][0], string_format)
    
writer.close()